# Case Study 8 - XGBoost, Random Forest and SVM

Allen Ansari, Chris Ballenger, Shantanu Godbole, Chad Madding

DS 7333 Quantifying the World

June 22, 2020

You have been given the following banking data.  There is no missing data:
https://smu.box.com/s/bizpcq4a3dtid04tke1x7nz7359r79y5
There is no metadata or data dictionary.  Your class target is labeled “target”

   1)	Build 3 tuned models: An XGBoost, a Random Forest, and an SVM.  

   2)	Show the log loss and accuracy for XGBoost, and Random Forest models on out of fold predictions.  Show the Accuracy of the SVM on a validation set  

   3)	Time how long it takes to do a sample of 1000, 2000, 5000, and 10,000 rows in the SVM.  What is the rough scaling of SVM with sample size?  


# Abstract  
Write abstract here.  

# Introduction
Write introduction here.  

# Methods and Results  
Write methods and results here.      

*The code used throughout the project is modified versions of  sample...*

### Using XGBoost in Python
First of all, just like what you do with any other dataset, you are going to import the dataset and store it in a variable called case8.

In [26]:
import pandas as pd

case_8 = pd.read_csv (r'data\case_8.csv')

case_8.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.630686,7.464411,C,4.145098,9.191265,2.436402,2.483921,2.301630,...,6.822439,3.549938,0.598896,AF,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.630686,7.464411,C,4.145098,8.742359,2.436402,2.483921,1.496569,...,6.822439,3.549938,0.919812,Z,1.672658,3.239542,2.030373,0,1.925763,1.739389


One Hot encoding of categorical features.

In [5]:
#case_8 = pd.get_dummies(case8)

In [18]:
# Get a list of columns with object
cols = case_8.dtypes
display(case_8.loc[:,cols == "object"])
columns = case_8.loc[:,cols == "object"].columns

,v3,v22,v24,v30,v31,v47,v52,v56,v66,v71,v74,v75,v79,v91,v107,v110,v112,v113,v125
0,C,XDX,C,C,A,C,G,DI,C,F,B,D,E,A,E,B,O,G,AU
1,C,GUV,C,C,A,E,G,DY,A,F,B,D,D,B,B,A,U,G,AF
2,C,FQ,E,C,A,C,F,AS,A,B,B,B,E,G,C,B,S,G,AE
3,C,ACUE,D,C,B,C,H,BW,A,F,B,D,B,B,B,B,J,G,CJ
4,C,HIT,E,C,A,I,H,BW,C,F,B,D,C,G,C,A,T,G,Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114316,C,ADQC,E,C,A,I,E,N,A,B,B,B,C,A,E,A,U,G,AL
114317,C,XRT,D,C,B,C,I,BW,A,F,B,D,B,G,C,B,J,G,E
114318,C,AGTG,A,C,B,C,D,BW,A,B,B,B,B,G,C,B,R,G,Q
114319,C,SGB,D,G,A,C,J,AS,A,F,B,D,E,A,E,B,A,G,BW


In [19]:
#One Hot Encode

from tqdm import tqdm

data = [ pd.get_dummies(case_8[col], prefix=col).copy() for col in tqdm(columns) ]
one_hot_df = pd.concat(data, axis=1)

display(one_hot_df.head())

100%|██████████| 19/19 [00:40<00:00,  2.11s/it]


,v3_A,v3_B,v3_C,v22_AA,v22_AAA,v22_AAAA,v22_AAAB,v22_AAAC,v22_AAAE,v22_AAAF,...,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
#Append back

#Drop old columns
case_8_ = case_8.drop(columns, axis=1)

 

case_8_final = pd.concat([case_8_, one_hot_df], axis=1)
case_8_ = None #take out of memory

 

print(case_8_final.shape)
display(case_8_final.head())

(114321, 18688)


,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,...,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z
0,3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,...,0,0,0,0,0,0,0,0,0,0
1,4,1,1.630686,7.464411,4.145098,9.191265,2.436402,2.483921,2.301630,9.031859,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,...,0,0,0,0,0,0,0,0,0,0
3,6,1,0.797415,8.304757,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,...,0,0,0,0,0,0,0,0,0,0
4,8,1,1.630686,7.464411,4.145098,8.742359,2.436402,2.483921,1.496569,9.031859,...,0,0,0,0,0,0,0,0,0,1


You can check for its keys using the .keys() method.

In [21]:
print (case_8_final.keys())

Index(['ID', 'target', 'v1', 'v2', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9',
       ...
       'v125_Q', 'v125_R', 'v125_S', 'v125_T', 'v125_U', 'v125_V', 'v125_W',
       'v125_X', 'v125_Y', 'v125_Z'],
      dtype='object', length=18688)


You can easily check for its shape by using the case8.shape attribute, which will return the size of the dataset.

In [22]:
print (case_8_final.shape)

(114321, 18688)


As you can see it returned (114321, 18688), that means there are 114321 rows of data with 18688 columns. The original data set contained 133 columns, after One Hot Encoding the count is now up to 18688.

Run the .info() method on your DataFrame to get useful information about the data.

In [23]:
case_8_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Columns: 18688 entries, ID to v125_Z
dtypes: float64(108), int64(6), uint8(18574)
memory usage: 2.1 GB


We will build the model using Trees as base learners (which are the default base learners) using XGBoost's scikit-learn compatible API. Along the way, we will also use some of the common tuning parameters which XGBoost provides in order to improve our model's performance, and using the root mean squared error (RMSE) performance metric to check the performance of the trained model on the test set. Root mean Squared error is the square root of the mean of the squared differences between the actual and the predicted values. We start by importing the library xgboost and other important libraries that we will be using for building the model.

In [10]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

Create a sparce matrix

In [11]:
# import sparse module from SciPy package 
from scipy import sparse
# import uniform module to create random numbers
from scipy.stats import uniform

In [24]:
case8_sparse = sparse.csr_matrix(case_8_final)

Separate the target variable and rest of the variables using .iloc to subset the data.

In [25]:
X, y = case8_sparse.iloc[:,:-1],case8_sparse.iloc[:,-1]

AttributeError: iloc not found

In [14]:
case8_dmatrix = xgb.DMatrix(data=X,label=y)

MemoryError: Unable to allocate 15.9 GiB for an array with shape (114321, 18687) and data type float64

Now, we will create the train and test set for cross-validation of the results using the train_test_split function from sklearn's model_selection module with test_size size equal to 20% of the data. Also, to maintain reproducibility of the results, a random_state is also assigned.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)